In [1]:
import numpy as np
import pandas as pd
import os

import chemprop


import joblib

In [2]:
np.random.seed(1234)

In [3]:
# endpoint = 'skin-sensitization'
endpoint = 'eye-irritation'

loc = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\data'
endpoint_loc = os.path.join(loc, endpoint)
model = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\src\models'
model_loc = os.path.join(model, endpoint)

In [4]:
train_filename = 'train.csv'
val_filename = 'val.csv'
arguments = [
    '--data_path', os.path.join(endpoint_loc, train_filename),
    '--dataset_type', 'classification',
    '--config_path', os.path.join(model_loc, 'config.json'),
    '--separate_test_path', os.path.join(endpoint_loc, val_filename), 
    '--save_dir', model_loc,
    '--target_columns', 'Activity', 
    '--smiles_columns', 'SMILES',
    '--features_generator', 'rdkit_2d_normalized', 
    '--no_features_scaling', 
    '--ensemble_size', '5', 
    '--extra_metrics', 'accuracy'
]

args = chemprop.args.TrainArgs().parse_args(arguments)
# chemprop.train.run_training(args=args)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

Command line
python c:\Users\kevin\nextAID\lib\site-packages\ipykernel_launcher.py --f=c:\Users\kevin\AppData\Roaming\jupyter\runtime\kernel-v2-235886Vxa3DwY3TcH.json
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': 'D:\\School\\Semester3\\Seminar - '
                'Reproducibili

In [5]:
val_filename = 'val.csv'
arguments = [
    '--test_path', os.path.join(endpoint_loc, val_filename), 
    '--preds_path', '/dev/null',
    '--checkpoint_dir', model_loc,
    '--smiles_columns', 'SMILES',
    '--features_generator', 'rdkit_2d_normalized', 
    '--no_features_scaling'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


776it [00:00, 129239.99it/s]
100%|██████████| 776/776 [00:45<00:00, 17.13it/s]


Validating SMILES
Test size = 775


  0%|          | 0/5 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".


...

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".


 80%|████████  | 4/5 [02:57<00:42, 42.39s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".


100%|██████████| 5/5 [03:38<00:00, 43.69s/it]

Saving predictions to /dev/null
Elapsed time = 0:04:24


In [6]:
preds

[[0.5945665657520294],
 [0.7223759055137634],
 [0.5087614178657531],
 [0.5976842999458313],
 [0.8043069005012512],
 [0.5421594381332397],
 [0.16182066947221757],
 [0.9534872770309448],
 [0.9069293737411499],
 [0.902154004573822],
 [0.1844962164759636],
 [0.37136169373989103],
 [0.9182733058929443],
 [0.8462374567985534],
 [0.34891530573368074],
 [0.8048105478286743],
 [0.9527966499328613],
 [0.6063166081905365],
 [0.12867302745580672],
 [0.9477950692176819],
 [0.920025908946991],
 [0.5724234938621521],
 [0.956162965297699],
 [0.45280042886734007],
 [0.9971102476119995],
 [0.8108995199203491],
 [0.6743675947189331],
 [0.4976506412029266],
 [0.3179577499628067],
 [0.23984797298908234],
 [0.9318285346031189],
 [0.4140376508235931],
 [0.8964445948600769],
 [0.8131973147392273],
 [0.6404035329818726],
 [0.8279972076416016],
 [0.6614622831344604],
 [0.7690639138221741],
 [0.9885819435119629],
 [0.9709149599075317],
 [0.5818681120872498],
 [0.23676041662693023],
 [0.016925808042287827],
 [0.7